In [ ]:
!pip install --upgrade pandas==0.25.3

In [ ]:
!pip install catboost

     |████████████████████████████████| 66.1MB 66kB/s 


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [ ]:
#import necessary dependecies
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier,VotingClassifier
 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
warnings.filterwarnings('ignore')
from typing import List
import tensorflow as tf
import random
from tqdm import tqdm 
import copy
 
tf.random.set_seed(111)
np.random.seed(111)
random.seed(111)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

In [ ]:
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_ = pd.read_csv('/content/drive/My Drive/ZimnatInsurance/Train.csv')
test_ = pd.read_csv('/content/drive/My Drive/ZimnatInsurance/Test.csv')
submission_ = pd.read_csv('/content/drive/My Drive/ZimnatInsurance/SampleSubmission.csv')

In [ ]:
from sklearn.model_selection import KFold

def get_train_test_names(train_, test_, submission_):
  kf = KFold(n_splits=5, shuffle=False)
  for r, (train_index, test_index) in enumerate(kf.split(train_)):
    test = train_.iloc[test_index]

    X_test = []
    X_test_columns = test.columns
    for v in test.values:
      info = v[:8]
      binary = v[8:]
      index = [k for k, i in enumerate(binary) if i == 1]
      for i in index:
        for k in range(len(binary)):
          if k == i:
            binary_transformed = list(copy.copy(binary))
            binary_transformed[i] = 0
            X_test.append(list(info) + binary_transformed)

    X_test = pd.DataFrame(X_test)
    X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']
    X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

    yield train_.iloc[train_index], X_test, submission_, '0_fold' + str(r) + '.csv'
  yield train_, test_, submission_, '0_main.csv'

### Get folds

In [ ]:
for train, test, submission, name in get_train_test_names(train_, test_, submission_):
  X_train = []
  X_train_columns = train.columns
  c = 0
  for v in train.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
      c+=1
      for k in range(len(binary)):
        if k == i:
          binary_transformed = list(copy.copy(binary))
          binary_transformed[i] = 0
          X_train.append(list(info) + binary_transformed + [X_train_columns[8+k]] + [c])

  X_train = pd.DataFrame(X_train)
  X_train.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'ID2']


  X_test = []
  true_values = []
  c = 0
  for v in test.values:
    c += 1
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    X_test.append(list(info) + list(binary) + [c])
    for k in test.columns[8:][index]:
      true_values.append(v[0] + ' X ' + k)

  X_test = pd.DataFrame(X_test)
  X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'ID2']


  features_train = []
  features_test = []
  columns = []

  append_features = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
  'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 
  'ECY3', 'ID', 'ID2', 'join_date', 'sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
  'birth_year']
  for v in append_features:
    features_train.append(X_train[v].values.reshape(-1, 1))
    features_test.append(X_test[v].values.reshape(-1, 1))
    columns.append(np.array([v]))

  y_train = X_train[['product_pred']]


  features_train = np.concatenate(features_train, axis=1)
  features_test = np.concatenate(features_test, axis=1)
  columns = np.concatenate(np.array(columns))

  X_train = pd.DataFrame(features_train)
  X_train.columns = columns
  X_test = pd.DataFrame(features_test)
  X_test.columns = columns
############################## fix code ##############################
  X_train.join_date = pd.to_datetime(X_train.join_date,)
  X_test.join_date = pd.to_datetime(X_test.join_date,)

  X_train.join_date = pd.to_datetime(X_train.join_date, format="%Y-%m-%d")
  X_test.join_date = pd.to_datetime(X_test.join_date, format="%Y-%m-%d")

  # new features

  X_train['day_of_week'] = X_train['join_date'].dt.dayofweek
  X_train['day_of_week_name'] = X_train['join_date'].dt.weekday_name
  X_train['age'] = np.abs(X_train['join_date'].dt.year - X_train['birth_year'])


  X_test['day_of_week'] = X_test['join_date'].dt.dayofweek
  X_test['day_of_week_name'] = X_test['join_date'].dt.weekday_name
  X_test['age'] = np.abs( X_test['join_date'].dt.year - X_test['birth_year'])

  #from catboost import CatBoostClassifier
  cat_features = ['branch_code','occupation_code', 'occupation_category_code']
        
  X_train = X_train.fillna(0)
  X_test = X_test.fillna(0)
  y_train = y_train.fillna(0)

  # LABEL ENCODE
  enc = LabelEncoder()
  def encode_LE(train,test,cols,verbose=True):
    for col in cols:
  
      df_comb = pd.concat([train[col].astype('str'),test[col].astype('str')],axis=0)
      df_comb = enc.fit_transform(df_comb)
      nm = col
      if df_comb.max()>32000: 
        train[nm] = df_comb[:len(train)].astype('int32')
        test[nm] = df_comb[len(train):].astype('int32')
      else:
        train[nm] = df_comb[:len(train)].astype('int16')
        test[nm] = df_comb[len(train):].astype('int16')
      del df_comb; x=gc.collect()
      if verbose: print(nm,', ',end='')

  X_train.day_of_week_name = X_train.day_of_week_name.astype('str')
  X_test.day_of_week_name = X_test.day_of_week_name.astype('str')
  X_train.join_date = X_train.join_date.astype('str')
  X_test.join_date = X_test.join_date.astype('str')

  data = X_train.append(X_test)
  for v in ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code','day_of_week_name','join_date']:
    data.loc[:,v] = enc.fit_transform(data.loc[:,v])
  X_train = data[:X_train.shape[0]]
  X_test = data[-X_test.shape[0]:]

  enc.fit(y_train.iloc[:,0])
  y_train = pd.DataFrame(enc.transform(y_train.iloc[:,0]))
  y_train.columns = ['target']

  X = X_train.drop(['ID','ID2','birth_year'], axis=1)
  test = X_test.drop(['ID','ID2','birth_year'], axis=1)
  df_train =X.values
  df_test =test.values
  y = y_train.target



############################### MODELING CODE : PAY attention :) ######################################
  import lightgbm as lgb
 
  sk = StratifiedKFold(n_splits= 5,random_state=111,shuffle=True)
  
  lgb_clf = lgb.LGBMClassifier(
      learning_rate = 0.015,
      n_estimators = 2000,
      objective ='multiclass',
      random_state = 111,
      num_leaves = 127,
      max_depth = 6,
      metric = 'multi_logloss'
  )

  lgb_predictions = list()

  for fold, (train_idx, test_idx) in enumerate(sk.split(df_train,y)):
    lgb_clf.fit(
        df_train[train_idx],
        y[train_idx],
        eval_set = [(df_train[test_idx], y[test_idx])],
        early_stopping_rounds  = 100,
        categorical_feature = [24,25,26] ,
        verbose=100 ,

    )
    preds = lgb_clf.predict_proba(df_test)
    lgb_predictions.append(preds)
  
  # get preds :D
  proba = np.average(lgb_predictions, axis=0)
  y_test = pd.DataFrame(proba)
  y_test.columns = enc.inverse_transform(y_test.columns)

  answer_mass = []
  for i in range(X_test.shape[0]):
    id = X_test['ID'].iloc[i]
    for c in y_test.columns:
      answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

  df_answer = pd.DataFrame(answer_mass)
  df_answer.columns = ['ID X PCODE', 'Label']
  for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
      df_answer['Label'].iloc[i] = 1.0

  df_answer.reset_index(drop=True, inplace=True)
  df_answer.to_csv(name, index=False)

[400]	valid_0's multi_logloss: 0.399304
[500]	valid_0's multi_logloss: 0.39041
[600]	valid_0's multi_logloss: 0.386566
[700]	valid_0's multi_logloss: 0.385257
[800]	valid_0's multi_logloss: 0.385225
Early stopping, best iteration is:
[740]	valid_0's multi_logloss: 0.385019
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's multi_logloss: 0.668146
[200]	valid_0's multi_logloss: 0.483062
[300]	valid_0's multi_logloss: 0.426267
[400]	valid_0's multi_logloss: 0.40409
[500]	valid_0's multi_logloss: 0.395846
[600]	valid_0's multi_logloss: 0.392799
[700]	valid_0's multi_logloss: 0.391822
[800]	valid_0's multi_logloss: 0.392643
Early stopping, best iteration is:
[713]	valid_0's multi_logloss: 0.391769
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's multi_logloss: 0.657126
[200]	valid_0's multi_logloss: 0.474477
[300]	valid_0's multi_logloss: 0.417945
[400]	valid_0's multi_logloss: 0.39654
[500]	valid_0's multi_logloss: 0.388837
[600]	v

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir '/content/drive/My Drive/ZimnatInsurance/Stack_LGBM_RONNY'

In [ ]:
!cp 0_fold0.csv  '/content/drive/My Drive/ZimnatInsurance/Stack_LGBM_RONNY' 
!cp 0_fold1.csv  '/content/drive/My Drive/ZimnatInsurance/Stack_LGBM_RONNY' 
!cp 0_fold2.csv  '/content/drive/My Drive/ZimnatInsurance/Stack_LGBM_RONNY' 
!cp 0_fold3.csv  '/content/drive/My Drive/ZimnatInsurance/Stack_LGBM_RONNY' 
!cp 0_fold4.csv  '/content/drive/My Drive/ZimnatInsurance/Stack_LGBM_RONNY' 
!cp 0_main.csv  '/content/drive/My Drive/ZimnatInsurance/Stack_LGBM_RONNY' 